In [65]:
import pandas as pd
import sys
#sys.path.append('../')
sys.path.append('/')
from utils_processing import transform_verbose_to_calendar, add_country_codes, format_days_to_ymwd
from utils_plot import plot_countries_been_over_years, create_travel_map

In [66]:
df_verbose = pd.read_csv('../data/trips_verbose.csv')
df = transform_verbose_to_calendar(df_verbose, save=False)
df = df[df['country'] != 'All']
#plot_countries_been_over_years(df)

In [67]:
VAR = 'total_days'
# create_travel_map(df, var=VAR, code_convention='code3', bins=[0, 7, 30, 365, df[VAR].max()], 
#                   save_path='../attachments/map_travel.html'
#                   #save_path=None
#                   )

In [74]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import matplotlib.colors as mcolors

def create_travel_globe(
    df_,
    var="total_days",
    code_convention="code3",
    bins=None,
    legend_labels=None,
    color=None,
    save_path=None
):

    # ---- Default legend labels ----
    if legend_labels is None:
        legend_labels = ["days", "weeks", "months", "years"]

    # ---- Ensure ISO codes ----
    df = add_country_codes(df_.copy())
    df = df.dropna(subset=[code_convention, var])

    # ---- Binning ----
    if bins is None:
        bins = np.linspace(df[var].min(), df[var].max(), 5)

    n_bins = len(bins) - 1
    df["bin"] = pd.cut(
        df[var],
        bins=bins,
        include_lowest=True,
        labels=False
    ).astype(int)

    legend_labels = legend_labels[:n_bins]

    # ---- Colors ----
    if color is None:
        # Default discrete Set1
        set1_colors = ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3"][:n_bins]
    elif isinstance(color, list):
        # Use provided discrete colors
        set1_colors = color[:n_bins]
    else:
        # Single color: create a log-scaled gradient
        # Convert base color to RGB
        base_rgb = np.array(mcolors.to_rgb(color))
        # Log scale based on bin midpoints
        bin_mids = np.array([bins[i]+(bins[i+1]-bins[i])/2 for i in range(n_bins)])
        log_norm = (np.log(bin_mids) - np.log(bin_mids.min())) / (np.log(bin_mids.max()) - np.log(bin_mids.min()))
        # Darker = larger values
        set1_colors = [mcolors.to_hex(base_rgb * (0.3 + 0.7*(1-log_val))) for log_val in log_norm]

    # ---- Discrete colorscale ----
    colorscale = []
    for i, c in enumerate(set1_colors):
        colorscale.append([i / n_bins, c])
        colorscale.append([(i + 1) / n_bins, c])

    # ---- Tooltip ----
    def _tooltip(row):
        return f"{row.get('country', row[code_convention])}: {format_days_to_ymwd(row[var])}"

    hover_text = df.apply(_tooltip, axis=1)

    # ---- Choropleth ----
    fig = go.Figure(
        go.Choropleth(
            locations=df[code_convention],
            z=df["bin"],
            locationmode="ISO-3",
            text=hover_text,
            hoverinfo="text",
            colorscale=colorscale,
            zmin=0,
            zmax=n_bins,
            marker_line_width=0,
            showscale=False
        )
    )

    fig.update_traces(
        hoverlabel=dict(
            bgcolor='black',
            )
        )

    # ---- Geo config ----
    fig.update_geos(
        projection_type="orthographic",
        #resolution=50,
        showland=True,
        landcolor="white",
        showocean=True,
        oceancolor="skyblue",
        showcountries=False,
        showcoastlines=True,
        showframe=False,
        bgcolor="black",
        coastlinecolor='white',
        coastlinewidth=0.5
    )

    # ---- Layout ----
    fig.update_layout(
        paper_bgcolor="black",
        plot_bgcolor="black",
        margin=dict(l=0, r=0, t=0, b=0),
        dragmode="pan",
        hovermode="closest",
    )

    if save_path:
        fig.write_html(save_path, include_plotlyjs="cdn")

    return fig

create_travel_globe(
    df, 
    var=VAR, 
    code_convention='code3', 
    bins=[0, 7, 30, 365, df[VAR].max()],
    legend_labels = ["days", "weeks", "months", "years"],
    color='royalblue',
    save_path='../attachments/map_travel.html'
    #save_path=None
    )